In [63]:
import torch

In [64]:
a = [1,2,3,4,5,6]
b = [0,1,2,3,4,5]

A = torch.tensor(a).reshape(2,3)
B = torch.tensor(b).reshape(3,2)

In [3]:
A

tensor([[1, 2, 3],
        [4, 5, 6]])

In [4]:
B

tensor([[0, 1],
        [2, 3],
        [4, 5]])

In [5]:
A@B

tensor([[16, 22],
        [34, 49]])

In [6]:
print(A[0, :] @ B)
print(A[1, :] @ B)

tensor([16, 22])
tensor([34, 49])


Expand B so that it repeats itself

In [7]:
A.shape

torch.Size([2, 3])

In [8]:
(A.reshape(2,3,1).broadcast_to(2,3,2)*B.reshape(1,3,2).broadcast_to(2,3,2)).sum(1)

tensor([[16, 22],
        [34, 49]])

In [9]:
import numpy as np
def custom_matmul(A, B):
    # Expand dimensions to prepare for broadcasting
    expanded_A = A[:, :, np.newaxis]  # Shape: (m, n, 1)
    expanded_B = B[np.newaxis, :, :]  # Shape: (1, n, p)
    
    # Element-wise multiplication (broadcasted)
    product = expanded_A * expanded_B  # Shape: (m, n, p)
    
    # Sum over the shared dimension (n)
    result = product.sum(axis=1)      # Shape: (m, p)
    
    return result

custom_matmul(np.array(a).reshape(2,3), np.array(b).reshape(3,2))

array([[16, 22],
       [34, 49]])

In [39]:
def compute_strides(shape):
	strides = [1 for _ in shape]
	for i in range(len(shape)-1, 0, -1):
		strides[i-1] = shape[i]*strides[i]
	return strides

def expand_start(shape: list[int], strides: list[int], expand_to: int):
	return [expand_to, *shape], [0, *strides]
def expand_end(shape: list[int], strides: list[int], expand_to: int):
	return [*shape, expand_to], [*strides, 0]


def manual_matmul(a, a_shape, b, b_shape):
	a_strides = compute_strides(a_shape)
	b_strides = compute_strides(b_shape)
	print("a", a, a_shape, a_strides)
	print("b", b, b_shape, b_strides)

	ae_shape, ae_strides = expand_end(a_shape, a_strides, a_shape[0])
	be_shape, be_strides = expand_start(b_shape, b_strides, b_shape[1])
	
	# elementwise add
	for i in range(ae_shape[0]):
		for k in range(ae_shape[2]):
			for j in range(ae_shape[1]):
				a_ijk = a[i*ae_strides[0] + j*ae_strides[1] + k*ae_strides[2]]
				b_ijk = a[i*be_strides[0] + j*be_strides[1] + k*be_strides[2]]
				print(a_ijk*b_ijk)

			
		

	

manual_matmul(a,[2,3], b, [3,2])

a [1, 2, 3, 4, 5, 6] [2, 3] [3, 1]
b [0, 1, 2, 3, 4, 5] [3, 2] [2, 1]
1
6
15
2
8
18
4
15
30
8
20
36


In [78]:
(A.reshape(2,3,1).broadcast_to((2, 3, 2))*B.reshape(1,3,2).broadcast_to((2,3,2)))

tensor([[[ 0,  1],
         [ 4,  6],
         [12, 15]],

        [[ 0,  4],
         [10, 15],
         [24, 30]]])

In [82]:
ae_shape = [2,3]
ae_strides = [3, 1]
for i in range(ae_shape[0]):
	for j in range(ae_shape[1]):
		print(a[i*ae_strides[0] + j*ae_strides[1]], end=" ")
	print("|")
print("|")

1 2 3 |
4 5 6 |
|


In [86]:
A.reshape(2,3,1).broadcast_to((2,3,2))

tensor([[[1, 1],
         [2, 2],
         [3, 3]],

        [[4, 4],
         [5, 5],
         [6, 6]]])

In [84]:
ae_shape = [2,3,2]
ae_strides = [3, 1, 0]
for i in range(ae_shape[0]):
	for j in range(ae_shape[1]):
		for k in range(ae_shape[2]):
			print(a[i*ae_strides[0] + j*ae_strides[1] + k*ae_strides[2]], end=" ")
		print("|k")
	print("|j")
print("|i")

1 1 |k
2 2 |k
3 3 |k
|j
4 4 |k
5 5 |k
6 6 |k
|j
|i


In [87]:
B.reshape(1,3, 2).broadcast_to((2,3,2))

tensor([[[0, 1],
         [2, 3],
         [4, 5]],

        [[0, 1],
         [2, 3],
         [4, 5]]])

In [89]:
be_shape = [2,3,2]
be_strides = [0, 2, 1]
for i in range(be_shape[0]):
	for j in range(be_shape[1]):
		for k in range(be_shape[2]):
			print(b[i*be_strides[0] + j*be_strides[1] + k*be_strides[2]], end=" ")
		print("|k")
	print("|j")
print("|i")

0 1 |k
2 3 |k
4 5 |k
|j
0 1 |k
2 3 |k
4 5 |k
|j
|i


In [95]:
(A.reshape(2,3,1).broadcast_to((2, 3, 2))*B.reshape(1,3,2).broadcast_to((2,3,2)))

tensor([[[ 0,  1],
         [ 4,  6],
         [12, 15]],

        [[ 0,  4],
         [10, 15],
         [24, 30]]])

In [90]:
be_shape = [2,3,2]
be_strides = [0, 2, 1]
for i in range(be_shape[0]):
	for j in range(be_shape[1]):
		for k in range(be_shape[2]):
			print(a[i*ae_strides[0] + j*ae_strides[1] + k*ae_strides[2]]*b[i*be_strides[0] + j*be_strides[1] + k*be_strides[2]], end=" ")
		print("|k")
	print("|j")
print("|i")

0 1 |k
4 6 |k
12 15 |k
|j
0 4 |k
10 15 |k
24 30 |k
|j
|i


In [93]:
be_shape = [2,3,2]
be_strides = [0, 2, 1]
for i in range(be_shape[0]):
	for k in range(be_shape[2]):
		summed = 0
		for j in range(be_shape[1]):
			summed += a[i*ae_strides[0] + j*ae_strides[1] + k*ae_strides[2]]*b[i*be_strides[0] + j*be_strides[1] + k*be_strides[2]]
		print(summed)
	print("|k")
print("|i")

16
22
|k
34
49
|k
|i


In [97]:
np.array(a).reshape((2,3)).T@np.array(b).reshape((2,3))

array([[12, 17, 22],
       [15, 22, 29],
       [18, 27, 36]])